In [1]:
import pandas as pd

In [118]:
df_bronx = pd.read_csv('PLUTO/BX_18v1.csv', low_memory=False)

In [119]:
print ("Bronx data shape:\n", df_bronx.shape)
print ("\n\nBronx data columns:\n", df_bronx.columns)
print ("\n\nBronx data types:\n", df_bronx.dtypes)
print ("\n\nBronx data nulls:\n", df_bronx.isnull().sum())
df_bronx.head()

Bronx data shape:
 (89854, 87)


Bronx data columns:
 Index(['Borough', 'Block', 'Lot', 'CD', 'CT2010', 'CB2010', 'SchoolDist',
       'Council', 'ZipCode', 'FireComp', 'PolicePrct', 'HealthCenterDistrict',
       'HealthArea', 'SanitBoro', 'SanitDistrict', 'SanitSub', 'Address',
       'ZoneDist1', 'ZoneDist2', 'ZoneDist3', 'ZoneDist4', 'Overlay1',
       'Overlay2', 'SPDist1', 'SPDist2', 'SPDist3', 'LtdHeight', 'SplitZone',
       'BldgClass', 'LandUse', 'Easements', 'OwnerType', 'OwnerName',
       'LotArea', 'BldgArea', 'ComArea', 'ResArea', 'OfficeArea', 'RetailArea',
       'GarageArea', 'StrgeArea', 'FactryArea', 'OtherArea', 'AreaSource',
       'NumBldgs', 'NumFloors', 'UnitsRes', 'UnitsTotal', 'LotFront',
       'LotDepth', 'BldgFront', 'BldgDepth', 'Ext', 'ProxCode', 'IrrLotCode',
       'LotType', 'BsmtCode', 'AssessLand', 'AssessTot', 'ExemptLand',
       'ExemptTot', 'YearBuilt', 'YearAlter1', 'YearAlter2', 'HistDist',
       'Landmark', 'BuiltFAR', 'ResidFAR', 'CommFAR',

,Borough,Block,Lot,CD,CT2010,CB2010,SchoolDist,Council,ZipCode,FireComp,...,ZMCode,Sanborn,TaxMap,EDesigNum,APPBBL,APPDate,PLUTOMapID,FIRM07_FLAG,PFIRM15_FLAG,Version
0,BX,2260,1,201,19.0,1022.0,7.0,8.0,10454.0,L029,...,NaN,209S016,20901.0,E-143,0.0,NaN,1,NaN,NaN,18V1
1,BX,2260,4,201,19.0,1022.0,7.0,8.0,10454.0,L029,...,NaN,209S016,20901.0,E-143,0.0,NaN,1,NaN,NaN,18V1
2,BX,2260,10,201,19.0,1022.0,7.0,8.0,10454.0,L029,...,NaN,209S016,20901.0,E-143,0.0,NaN,1,NaN,NaN,18V1
3,BX,2260,17,201,19.0,1022.0,7.0,8.0,10454.0,L029,...,NaN,209S016,20901.0,E-143,0.0,NaN,1,NaN,NaN,18V1
4,BX,2260,18,201,19.0,1022.0,7.0,8.0,10454.0,L029,...,NaN,209S016,20901.0,E-143,0.0,NaN,1,NaN,NaN,18V1


## 2. Step

Reduce datasets to what might be relevant to heating problems.

In [120]:
df_bronx = df_bronx[['Borough', 'Address', 'NumBldgs', 'NumFloors', 'UnitsRes', 'UnitsTotal',
                    'YearBuilt', 'YearAlter1', 'YearAlter2']]

## 3. Step

Reduce complaints to addresses

In [ ]:
df_311 = pd.read_csv("fhrw-4uyv.csv", parse_dates=True)
df_heat = df_311[(df_311["complaint_type"] == "HEATING") | (df_311["complaint_type"] == "HEAT/HOT WATER")]

In [103]:
df_heat_adresses = df_heat.groupby(['borough','incident_address']).size().to_frame().reset_index()
df_heat_adresses.columns = ['Borough', 'Address', 'num_complaints']
df_heat_adresses["class_complaints"] = pd.cut(df_heat_adresses["num_complaints"], bins=[0,10,100,100000], labels=['low', 'medium', 'high'])

In [104]:
df_heat_adresses["num_complaints"].describe()

count    164461.000000
mean         12.276722
std          64.696463
min           1.000000
25%           1.000000
50%           3.000000
75%           8.000000
max       12360.000000
Name: num_complaints, dtype: float64

In [105]:
df_heat_adresses.sort_values(by=["num_complaints"], ascending=False).head(10)

,Borough,Address,num_complaints,class_complaints
79180,MANHATTAN,34 ARDEN STREET,12360,high
115617,QUEENS,89-21 ELMHURST AVENUE,9854,high
14637,BRONX,3810 BAILEY AVENUE,7109,high
317,BRONX,1025 BOYNTON AVENUE,5761,high
64639,BROOKLYN,9511 SHORE ROAD,4194,high
7726,BRONX,2090 EAST TREMONT AVENUE,3665,high
20893,BRONX,888 GRAND CONCOURSE,2980,high
11941,BRONX,2968 PERRY AVENUE,2763,high
88859,MANHATTAN,97 SHERMAN AVENUE,2521,high
7646,BRONX,2074 WALLACE AVENUE,2473,high


## 3. Step

Mark addresses with complaints.

In [123]:
df_bronx["Borough"] = 'BRONX'

In [127]:
print ("Number of addresses with complaints about in the Bronx: ", len(df_heat_adresses[df_heat_adresses["Borough"] == "BRONX"]["Address"].unique()))

Number of addresses with complaints about in the Bronx:  21976


In [129]:
print ("Rows with no address in Bronx dataset: ", df_bronx["Address"].isnull().sum())
df_bronx.dropna(subset=["Address"], inplace=True)

print ("Rows with no address in complaints dataset: ", df_heat_adresses["Address"].isnull().sum())
df_heat_adresses.dropna(subset=["Address"], inplace=True)


Rows with no address in Bronx dataset:  0
Rows with no address in complaints dataset:  0


In [130]:
df_bronx2 = pd.merge(df_bronx, df_heat_adresses, how='left', on=['Borough', 'Address'])

In [132]:
df_bronx2.head(20)

,Borough,Address,NumBldgs,NumFloors,UnitsRes,UnitsTotal,YearBuilt,YearAlter1,YearAlter2,num_complaints,class_complaints
0,BRONX,122 BRUCKNER BOULEVARD,1,0.0,0,0,0,0,0,NaN,NaN
1,BRONX,126 BRUCKNER BOULEVARD,2,1.0,0,1,1931,1994,0,NaN,NaN
2,BRONX,138 BRUCKNER BOULEVARD,1,2.0,0,1,1931,0,0,NaN,NaN
3,BRONX,144 BRUCKNER BOULEVARD,1,5.0,15,15,1931,2001,0,2.0,low
4,BRONX,148 BRUCKNER BOULEVARD,1,5.0,8,10,1920,2009,0,13.0,medium
5,BRONX,ST ANNS AVENUE,0,0.0,0,0,0,0,0,NaN,NaN
6,BRONX,519 EAST 132 STREET,2,1.0,0,2,1931,0,0,NaN,NaN
7,BRONX,517 EAST 132 STREET,1,3.0,0,1,1931,0,0,NaN,NaN
8,BRONX,LINCOLN AVENUE,0,0.0,0,0,0,0,0,NaN,NaN
9,BRONX,EAST 132 STREET,0,0.0,0,0,0,0,0,NaN,NaN
